# Model evaluation

The model performs image segmentation and output an image for each object tracked. The higher the value in the image, the more likely that the object was there. 

In [1]:
import torch
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from torch import optim
from datetime import datetime
import albumentations as A
import cv2

from unetTracker.trackingProject import TrackingProject
from unetTracker.multiClassUNetDataset import MultiClassUNetDataset
from unetTracker.unet import Unet

In [2]:
project = TrackingProject(name="faceTrack",root_folder = "/home/kevin/Documents/trackingProjects/")

Project directory: /home/kevin/Documents/trackingProjects/faceTrack
Loading /home/kevin/Documents/trackingProjects/faceTrack/config.yalm
{'augmentation_HorizontalFlipProb': 0.0, 'augmentation_RandomBrightnessContrastProb': 0.2, 'augmentation_RandomSizedCropProb': 1.0, 'augmentation_RotateProb': 0.3, 'image_size': [480, 640], 'labeling_ImageEnlargeFactor': 2.0, 'name': 'faceTrack', 'object_colors': [(0.0, 0.0, 255.0), (255.0, 0.0, 0.0), (255.0, 255.0, 0.0), (128.0, 0.0, 128.0)], 'objects': ['nose', 'chin', 'rEye', 'lEye'], 'target_radius': 10}


In [8]:
TRAIN_IMG_DIR ="/home/kevin/Documents/trackingProjects/faceTrack/dataset/train_images"
TRAIN_MASK_DIR ="/home/kevin/Documents/trackingProjects/faceTrack/dataset/train_masks" 
TRAIN_COORDINATES_DIR ="/home/kevin/Documents/trackingProjects/faceTrack/dataset/train_coordinates" 
VAL_IMG_DIR = "/home/kevin/Documents/trackingProjects/faceTrack/dataset/val_images"
VAL_MASK_DIR ="/home/kevin/Documents/trackingProjects/faceTrack/dataset/val_masks"
VAL_COORDINATES_DIR ="/home/kevin/Documents/trackingProjects/faceTrack/dataset/val_coordinates"
DEVICE = (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")) 
BATCH_SIZE=4
NUM_WORKERS=2
PIN_MEMORY=True

In [9]:
trainDataset = MultiClassUNetDataset(TRAIN_IMG_DIR, TRAIN_MASK_DIR,TRAIN_COORDINATES_DIR, transform=None)
valDataset = MultiClassUNetDataset(VAL_IMG_DIR, VAL_MASK_DIR,VAL_COORDINATES_DIR, transform=None)
trainLoader = DataLoader(trainDataset,shuffle=True,batch_size=BATCH_SIZE, num_workers=NUM_WORKERS,pin_memory=PIN_MEMORY)
valLoader = DataLoader(valDataset,shuffle=False,batch_size=BATCH_SIZE, num_workers=NUM_WORKERS,pin_memory = PIN_MEMORY)

In [10]:
BATCH_SIZE=2
trainLoader = DataLoader(trainDataset,
                          shuffle=True,
                          batch_size=BATCH_SIZE,
                          num_workers=4)
valLoader = DataLoader(valDataset,
                          shuffle=False,
                          batch_size=BATCH_SIZE,
                          num_workers=4)

In [13]:
model = Unet(in_channels=3, out_channels=len(project.object_list)).to(DEVICE)
project.load_model(model)
